In [2179]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from astral import LocationInfo
from astral.sun import sun
import pandas as pd
from astral.sun import elevation
from datetime import timedelta

In [2180]:
#df = pd.read_csv("Fallvilt_beriket_med_vær.csv",sep=";", low_memory=False)
#df = pd.read_csv('Fallvilt_månedsberiket.csv', sep=";", low_memory=False)
df = pd.read_csv('Fallvilt_tidspunkter.csv', sep=";", low_memory=False)

In [2181]:
df=df[df['Art'].isin(['Elg', 'Hjort', 'Rådyr'])].copy()  
#df=df[df['Art'].isin(['Hjort'])].copy()  
df=df[df['vegkategori'].isin(['E','F','K'])].copy()
df=df[df['År']==2025].copy()

In [2182]:
df= df[df['UkjentTidspunkt']==False].copy()

In [2183]:
# Representativ plassering for Trøndelag
TRONDELAG = LocationInfo(
    name="Trøndelag",
    region="Norway",
    timezone="Europe/Oslo",
    latitude=63.4,
    longitude=10.4,
)


In [2184]:
def solhøgde(ts, clip=(-12, 12)):
    solhogde = np.round(elevation(TRONDELAG.observer, ts),0)
    #solhogde = np.around(solhogde / 5) * 5
    #h_clip= np.clip(solhogde, clip[0], clip[1])
    return solhogde

In [2185]:
def lyskategori_fra_tidspunkt(ts):
    if pd.isna(ts):
        return None

    # Bruk solhøgde (grader over/under horisont)
    solhoyde = elevation(TRONDELAG.observer, ts)
    if solhoyde > 12:
        return "Dag"
    #elif solhoyde >= 0:
    #    return "Låg sol"
    elif solhoyde >-12:
        return "Skumring"
    else:
        return "Natt"

In [2186]:
df.columns

Index(['Dato', 'År', 'Kommune', 'Stedfesting', 'Art', 'Kjønn', 'Alder',
       'Årsak', 'Utfall', 'Merkelappnummer', 'Fallvilt-ID', 'UTM33 øst',
       'UTM33 nord', 'vegsystemreferanse.kortform', 'vegkategori', 'fase',
       'vegnr', 'strekning', 'delstrekning', 'arm', 'adskilte_løp',
       'trafikantgruppe', 'retning', 'meter', 'veglenkesekvensid',
       'relativPosisjon', 'veglenkesekvens.kortform', 'geometri.wkt',
       'geometri.srid', 'kommune (treff)', 'avstand_vegnettet_m',
       'Vegobjekt_540_id', 'ÅDT, total', 'Vegobjekt_105_id', 'Fartsgrense',
       'Vegobjekt_540_lengde', 'snow_depth', 'max_temperature',
       'min_temperature', 'mean_temperature', 'total_precipitation',
       'max_wind_speed', 'mean_wind_speed', 'max_wind_gust',
       'precipitation_type', 'weather_station_id', 'Måned',
       'monthly_snow_depth', 'monthly_mean_temperature',
       'monthly_mean_wind_speed', 'HendelsesDatoTid', 'UkjentTidspunkt'],
      dtype='object')

In [2187]:
df=df[['Vegobjekt_105_id','HendelsesDatoTid','ÅDT, total','Vegobjekt_540_lengde']].dropna().copy()

In [2188]:
df["HendelsesDatoTid"] = pd.to_datetime(df["HendelsesDatoTid"]).copy()
if df["HendelsesDatoTid"].dt.tz is None:
    df["HendelsesDatoTid"] = df["HendelsesDatoTid"].dt.tz_localize("Europe/Oslo")

In [2189]:
def maaned_til_arstid(m):
    if m in [12, 1, 2]:
        return "Vinter"
    elif m in [3, 4, 5]:
        return "Vår"
    elif m in [6, 7, 8]:
        return "Sommar"
    else:
        return "Haust"

In [2190]:
df["årstid"] = df["HendelsesDatoTid"].dt.month.apply(maaned_til_arstid)
df["årstid"] = df["årstid"].astype("category").copy()

In [2191]:
df["lyskategori"] = df["HendelsesDatoTid"].apply(lyskategori_fra_tidspunkt)
df["lyskategori"] = df["lyskategori"].astype("category")

In [2192]:
df

,Vegobjekt_105_id,HendelsesDatoTid,"ÅDT, total",Vegobjekt_540_lengde,årstid,lyskategori
231,135094429.0,2025-12-31 04:55:00+01:00,5630.0,9450.411,Vinter,Natt
234,85311783.0,2025-12-31 10:00:00+01:00,640.0,3192.223,Vinter,Skumring
236,85288934.0,2025-12-30 16:30:00+01:00,90.0,4153.251,Vinter,Skumring
237,86974451.0,2025-12-30 21:20:00+01:00,700.0,17818.619,Vinter,Natt
241,85310248.0,2025-12-30 06:13:00+01:00,300.0,3913.985,Vinter,Natt
...,...,...,...,...,...,...
2934,843859022.0,2025-01-01 18:11:00+01:00,1700.0,3122.634,Vinter,Natt
2935,728125448.0,2025-01-01 18:50:00+01:00,1600.0,11320.552,Vinter,Natt
2936,86975513.0,2025-01-01 16:55:00+01:00,800.0,12995.376,Vinter,Skumring
2937,86973808.0,2025-01-01 08:15:00+01:00,1210.0,2278.729,Vinter,Skumring


In [2193]:
df["solhogde"] = df["HendelsesDatoTid"].apply(solhøgde)

In [2194]:
df

,Vegobjekt_105_id,HendelsesDatoTid,"ÅDT, total",Vegobjekt_540_lengde,årstid,lyskategori,solhogde
231,135094429.0,2025-12-31 04:55:00+01:00,5630.0,9450.411,Vinter,Natt,-30.0
234,85311783.0,2025-12-31 10:00:00+01:00,640.0,3192.223,Vinter,Skumring,-0.0
236,85288934.0,2025-12-30 16:30:00+01:00,90.0,4153.251,Vinter,Skumring,-9.0
237,86974451.0,2025-12-30 21:20:00+01:00,700.0,17818.619,Vinter,Natt,-40.0
241,85310248.0,2025-12-30 06:13:00+01:00,300.0,3913.985,Vinter,Natt,-21.0
...,...,...,...,...,...,...,...
2934,843859022.0,2025-01-01 18:11:00+01:00,1700.0,3122.634,Vinter,Natt,-19.0
2935,728125448.0,2025-01-01 18:50:00+01:00,1600.0,11320.552,Vinter,Natt,-23.0
2936,86975513.0,2025-01-01 16:55:00+01:00,800.0,12995.376,Vinter,Skumring,-11.0
2937,86973808.0,2025-01-01 08:15:00+01:00,1210.0,2278.729,Vinter,Skumring,-9.0


In [2195]:
df["eksponering"] = (
    df["ÅDT, total"]
    * 365
    * df["Vegobjekt_540_lengde"]
    / 100000
)

df["log_eksponering"] = np.log(df["eksponering"])


In [2196]:
df

,Vegobjekt_105_id,HendelsesDatoTid,"ÅDT, total",Vegobjekt_540_lengde,årstid,lyskategori,solhogde,eksponering,log_eksponering
231,135094429.0,2025-12-31 04:55:00+01:00,5630.0,9450.411,Vinter,Natt,-30.0,194201.220844,12.176650
234,85311783.0,2025-12-31 10:00:00+01:00,640.0,3192.223,Vinter,Skumring,-0.0,7457.032928,8.916913
236,85288934.0,2025-12-30 16:30:00+01:00,90.0,4153.251,Vinter,Skumring,-9.0,1364.342954,7.218428
237,86974451.0,2025-12-30 21:20:00+01:00,700.0,17818.619,Vinter,Natt,-40.0,45526.571545,10.726051
241,85310248.0,2025-12-30 06:13:00+01:00,300.0,3913.985,Vinter,Natt,-21.0,4285.813575,8.363066
...,...,...,...,...,...,...,...,...,...
2934,843859022.0,2025-01-01 18:11:00+01:00,1700.0,3122.634,Vinter,Natt,-19.0,19375.943970,9.871788
2935,728125448.0,2025-01-01 18:50:00+01:00,1600.0,11320.552,Vinter,Natt,-23.0,66112.023680,11.099106
2936,86975513.0,2025-01-01 16:55:00+01:00,800.0,12995.376,Vinter,Skumring,-11.0,37946.497920,10.543932
2937,86973808.0,2025-01-01 08:15:00+01:00,1210.0,2278.729,Vinter,Skumring,-9.0,10064.006628,9.216721


In [2197]:
df_agg = (
    df
    .groupby(
        ["Vegobjekt_105_id", "årstid", "solhogde"],
        observed=True,      # fjern FutureWarning
        as_index=False
    )
    .agg(
        antall_kollisjoner=("Vegobjekt_105_id", "count"),
        log_eksponering=("log_eksponering", "first")
    )
)

In [2198]:
df_agg.count()

Vegobjekt_105_id      1966
årstid                1966
solhogde              1966
antall_kollisjoner    1966
log_eksponering       1966
dtype: int64

In [2199]:
# 1. Summen skal stemme
df_agg["antall_kollisjoner"].sum() == len(df)

True

In [2200]:
# 2. Ingen NaN
df_agg.isna().sum()

Vegobjekt_105_id      0
årstid                0
solhogde              0
antall_kollisjoner    0
log_eksponering       0
dtype: int64

In [2201]:
# 3. Fornuftige gruppestorleikar
df_agg["antall_kollisjoner"].describe()

count    1966.000000
mean        1.021363
std         0.151503
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: antall_kollisjoner, dtype: float64

In [2202]:
#f_agg=df_agg[df_agg['antall_kollisjoner']<10].copy()

In [2217]:
model_nb = smf.glm(
    formula="antall_kollisjoner ~ solhogde+C(årstid)+solhogde:C(årstid)",
    data=df_agg,
    family=sm.families.NegativeBinomial(alpha=2),
    #family=sm.families.Poisson(),
    offset=df_agg["log_eksponering"]
).fit()

print(model_nb.summary())



                 Generalized Linear Model Regression Results                  
Dep. Variable:     antall_kollisjoner   No. Observations:                 1966
Model:                            GLM   Df Residuals:                     1958
Model Family:        NegativeBinomial   Df Model:                            7
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3794.4
Date:                Wed, 04 Feb 2026   Deviance:                       1061.0
Time:                        15:56:31   Pearson chi2:                 2.52e+04
No. Iterations:                     9   Pseudo R-squ. (CS):           0.004111
Covariance Type:            nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [2204]:
import numpy as np

def lag_arstidsjustering(model):
    """
    Lag justeringsfaktorar for årstid frå ein statsmodels GLM (NB / Poisson).

    Referansekategori får faktor 1.0.
    """
    params = model.params

    # Finn alle årstids-koeffisientar
    arstid_params = {
        k: v for k, v in params.items()
        if k.startswith("C(årstid)")
    }

    # Referanse (den som ikkje er i params)
    arstid_justering = {"Haust": 1.0}

    # Legg til dei estimerte årstidene
    for k, beta in arstid_params.items():
        # Trekk ut årstidsnamn, t.d. C(årstid)[T.Sommar] -> Sommar
        arstid = k.split("[T.")[1].rstrip("]")
        arstid_justering[arstid] = float(np.exp(beta))

    return arstid_justering


In [2205]:
ARSTID_JUSTERING = lag_arstidsjustering(model_nb)


In [2206]:
ARSTID_JUSTERING

{'Haust': 1.0,
 'Sommar': 0.8387611461312544,
 'Vinter': 0.9663227350322376,
 'Vår': 0.8946449121739044}

In [2207]:
def lag_lysjustering(model, referanse="Dag", damping=1, normaliser=False):
    """
    Lag justeringsfaktorar for lysforhold frå GLM,
    med valfri damping for å unngå overtolking.
    """

    params = model.params
    lys_justering = {referanse: 1.0}

    for k, beta in params.items():
        if k.startswith("C(lyskategori)[T."):
            lys = k.split("[T.")[1].rstrip("]")
            # Demp effekten
            beta_dempet = damping * beta
            lys_justering[lys] = float(np.exp(beta_dempet))

    if normaliser:
        mean_factor = np.mean(list(lys_justering.values()))
        lys_justering = {
            k: v / mean_factor for k, v in lys_justering.items()
        }

    return lys_justering

In [2208]:
LYSJUSTERING=lag_lysjustering(model_nb)

In [2209]:
LYSJUSTERING

{'Dag': 1.0}

In [2210]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .sort_values(ascending=False)
)


årstid
Haust     657
Vinter    625
Vår       377
Sommar    349
Name: antall_kollisjoner, dtype: int64

In [2211]:
(
    df
    .groupby("årstid", observed=True)
    .size()
    .sort_values(ascending=False)
)


årstid
Haust     657
Vinter    625
Vår       377
Sommar    349
dtype: int64

In [2212]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .pipe(lambda s: s / s.sum())
)


årstid
Haust     0.327191
Sommar    0.173805
Vinter    0.311255
Vår       0.187749
Name: antall_kollisjoner, dtype: float64

In [2213]:
(
    df
    .groupby("årstid", observed=True)
    .apply(
        lambda g: g.shape[0] / g["eksponering"].sum()
    )
    .sort_values(ascending=False)
)


C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24832\484447156.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


årstid
Haust     0.000022
Vinter    0.000020
Vår       0.000020
Sommar    0.000020
dtype: float64